Placing Orders and Executing Trades

In [2]:
import pandas as pd
import numpy as np
import tpqoa
from datetime import datetime, timedelta, timezone
import json

In [4]:
# start_time = datetime.now(timezone.utc) - timedelta(days = 5)
# end_time = datetime.now(timezone.utc)

In [6]:
df = pd.read_csv('oanda.cfg')

In [58]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, config_file, instrument, bar_length, window, units):
        super().__init__(config_file)
        self.instrument = instrument
        self.bar_length = bar_length
        self.tick_data = pd.DataFrame()
        self.raw_data = pd.DataFrame()
        self.position = 0
        self.units = units
        self.window = window

        try:
            account_summary = self.get_account_summary()
            print("Account Summary:", account_summary)
        except Exception as e:
            print("Error retrieving account summary:", e)

        # Call the method to fetch historical data
        self.get_historical_data()

    def get_historical_data(self):
        """Fetch historical data from OANDA with proper error handling."""
        # Set the time range (last 30 minutes)
        end_time = datetime.now(timezone.utc)
        start_time = end_time - timedelta(minutes=30)

        # Format timestamps to OANDA's requirements (YYYY-MM-DDTHH:MM:SSZ)
        start_str = start_time.strftime('%Y-%m-%dT%H:%M:%SZ')
        end_str = end_time.strftime('%Y-%m-%dT%H:%M:%SZ')

        try:
            params = {
                'price': 'M',           # Midpoint price
                'granularity': self.bar_length,
                'from': start_str,      # Using 'from' instead of 'start'
                'to': end_str           # Using 'to' instead of 'end'
            }
            
            print(f"Requesting data with parameters: {params}")
            response = self.get_history(instrument=self.instrument, **params)

            if 'candles' in response:
                print("Data retrieved successfully")
                self.tick_data = pd.DataFrame(response['candles'])
                self.process_data()
                return
            else:
                print(f"No candles in response. Response contains: {response}")

        except Exception as e:
            print(f"Error retrieving data: {e}")
            if hasattr(e, 'response') and hasattr(e.response, 'json'):
                try:
                    error_details = e.response.json()
                    print(f"API Error Details: {error_details}")
                except:
                    pass

        print("Failed to retrieve historical data")
        return None

    def process_data(self):
        """Process the fetched historical data."""
        try:
            if isinstance(self.tick_data, pd.DataFrame) and self.tick_data.empty:
                print("No data to process")
                return

            # Convert to DataFrame if it's not already
            if not isinstance(self.tick_data, pd.DataFrame):
                self.tick_data = pd.DataFrame(self.tick_data)

            # Create a copy of the data for processing
            processed_data = self.tick_data.copy()

            # Extract time and set as index
            if 'time' in processed_data.columns:
                processed_data['time'] = pd.to_datetime(processed_data['time'])
                processed_data.set_index('time', inplace=True)

            # Extract mid prices if available
            if 'mid' in processed_data.columns:
                mid_data = pd.json_normalize(processed_data['mid'])
                processed_data[['open', 'high', 'low', 'close']] = mid_data[['o', 'h', 'l', 'c']]
            
            # Convert price columns to float
            for col in ['open', 'high', 'low', 'close']:
                if col in processed_data.columns:
                    processed_data[col] = processed_data[col].astype(float)

            self.tick_data = processed_data
            
            print("\nProcessed Data Summary:")
            print(f"Number of bars: {len(processed_data)}")
            if not processed_data.empty:
                print(f"Date range: {processed_data.index.min()} to {processed_data.index.max()}")
                print("\nFirst few rows of processed data:")
                print(processed_data.head())

        except Exception as e:
            print(f"Error processing data: {str(e)}")

    def __repr__(self):
        return f"ConTrader(instrument='{self.instrument}', bar_length='{self.bar_length}', units={self.units})"

In [60]:
dir(tpqoa.tpqoa)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_stream_data_failsafe_thread',
 'cancel_order',
 'create_order',
 'get_account_summary',
 'get_history',
 'get_instruments',
 'get_positions',
 'get_prices',
 'get_transaction',
 'get_transactions',
 'on_success',
 'print_transactions',
 'retrieve_data',
 'stream_data',
 'stream_data_failsafe',
 'transform_datetime']

In [64]:
help(tpqoa.get_history)

AttributeError: module 'tpqoa' has no attribute 'get_history'

In [68]:
tpqoa_instance = tpqoa.tpqoa('oanda.cfg')
print(dir(tpqoa_instance))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_stream_data_failsafe_thread', 'access_token', 'account_id', 'account_type', 'cancel_order', 'config', 'create_order', 'ctx', 'ctx_stream', 'get_account_summary', 'get_history', 'get_instruments', 'get_positions', 'get_prices', 'get_transaction', 'get_transactions', 'hostname', 'on_success', 'print_transactions', 'retrieve_data', 'stop_stream', 'stream_data', 'stream_data_failsafe', 'stream_hostname', 'suffix', 'transform_datetime']


In [62]:
trader = ConTrader(
    config_file="oanda.cfg",
    instrument="EUR_USD",
    bar_length="M1",
    window=1,
    units=5000
)

Account Summary: {'id': '101-001-29655670-001', 'alias': 'Primary', 'currency': 'USD', 'balance': '99880.484', 'createdByUserID': 29655670, 'createdTime': '2024-08-08T00:09:15.700565792Z', 'guaranteedStopLossOrderMode': 'DISABLED', 'pl': '-52.77', 'resettablePL': '-52.77', 'resettablePLTime': '0', 'financing': '-66.746', 'commission': '0.0', 'guaranteedExecutionFees': '0.0', 'marginRate': '0.02', 'openTradeCount': 1, 'openPositionCount': 1, 'pendingOrderCount': 0, 'hedgingEnabled': False, 'unrealizedPL': '-688.37', 'NAV': '99192.114', 'marginUsed': '410.2556', 'marginAvailable': '98783.9484', 'positionValue': '20512.78', 'marginCloseoutUnrealizedPL': '-686.28', 'marginCloseoutNAV': '99194.204', 'marginCloseoutMarginUsed': '410.2556', 'marginCloseoutPercent': '0.00207', 'marginCloseoutPositionValue': '20512.78', 'withdrawalLimit': '98783.9484', 'marginCallMarginUsed': '410.2556', 'marginCallPercent': '0.00414', 'lastTransactionID': '121'}
Requesting data with parameters: {'price': 'M', 

In [14]:
trader.position

0

In [16]:
trader.units

5000

In [18]:
trader.get_positions()

[{'instrument': 'EUR_USD',
  'pl': '-52.77',
  'unrealizedPL': '-688.37',
  'marginUsed': '410.2556',
  'resettablePL': '-52.77',
  'financing': '-66.746',
  'commission': '0.0',
  'guaranteedExecutionFees': '0.0',
  'long': {'units': '19000.0',
   'averagePrice': 1.11574,
   'tradeIDs': ['42'],
   'pl': '-16.83',
   'unrealizedPL': '-688.37',
   'resettablePL': '-16.83',
   'financing': '-66.9039',
   'guaranteedExecutionFees': '0.0'},
  'short': {'units': '0.0',
   'pl': '-35.94',
   'unrealizedPL': '0.0',
   'resettablePL': '-35.94',
   'financing': '0.1579',
   'guaranteedExecutionFees': '0.0'}}]